In [ ]:
%%capture
import os

from datetime import datetime
from pathlib import Path

import pandas as pd
from dj_notebook import activate

env_file = os.environ["META_ENV"]
reports_folder = Path(os.environ["META_REPORTS_FOLDER"])
analysis_folder = Path(os.environ["META_ANALYSIS_FOLDER"])
pharmacy_folder = Path(os.environ["META_PHARMACY_FOLDER"])
igh_folder = Path(os.environ["IGH_FOLDER"])
plus = activate(dotenv_file=env_file)
pd.set_option("future.no_silent_downcasting", True)

In [ ]:
from edc_pdutils.dataframes import get_subject_visit
from edc_appointment.analytics import get_appointment_df
from edc_visit_schedule.models import SubjectScheduleHistory
from django_pandas.io import read_frame
from edc_visit_schedule.site_visit_schedules import site_visit_schedules
from tabulate import tabulate



In [ ]:
df_appt = get_appointment_df()
df_visit = get_subject_visit("meta_subject.subjectvisit")
now = datetime(2025, 11, 21, 0, 0, 0)
later = datetime(2025, 12, 31, 0, 0, 0)


In [ ]:
len(df_appt.query("appt_datetime>=@now and appt_datetime<=@later"))

In [ ]:
read_frame(SubjectScheduleHistory.objects.all())

glucose = []
schedule = site_visit_schedules.get_visit_schedule("visit_schedule").schedules.get("schedule")
for visit_code, visit in schedule.visits.items():
    models = list(set([f.model for f in visit.crfs if
                       'glucose' in f.model and 'glucosefbg' not in f.model and f.required]))
    if models:
        glucose.append(visit_code)
glucosefbg = []
for visit_code, visit in schedule.visits.items():
    models = list(set([f.model for f in visit.crfs if
                       'glucosefbg' in f.model and f.required]))
    if models:
        glucosefbg.append(visit_code)
# results = {v: "glucosefbg" for v in results}

date_from = datetime(2025, 11, 21, 0, 0, 0)
date_to = datetime(2025, 12, 31, 0, 0, 0)
cond_str = "appt_datetime>=@date_from and appt_datetime<=@date_to"

df_glucose = pd.Series(glucose, name="visit_code").to_frame()
df_glucose["visit_code"] = df_glucose["visit_code"].astype("Float64")
value_counts = \
df_glucose.merge(df_appt.query(cond_str)["visit_code"], on="visit_code", how="left")[
    "visit_code"].value_counts()
total = value_counts.sum()

print(f'from {date_from.strftime("%Y-%m-%d")} to {date_to.strftime("%Y-%m-%d")}')
print(tabulate(value_counts.sort_values(ascending=True).to_frame(), headers="keys", tablefmt="fancy_grid"))
print(f"Total = {total}")



In [ ]:
df_glucosefbg = pd.Series(glucosefbg, name="visit_code").to_frame()
df_glucosefbg["visit_code"] = df_glucosefbg["visit_code"].astype("Float64")
value_counts = \
df_glucosefbg.merge(df_appt.query(cond_str)["visit_code"], on="visit_code", how="left")[
    "visit_code"].value_counts()
total = value_counts.sum()

print(f'from {date_from.strftime("%Y-%m-%d")} to {date_to.strftime("%Y-%m-%d")}')
print(tabulate(value_counts.sort_values(ascending=True).to_frame(), headers="keys", tablefmt="fancy_grid"))
print(f"Total = {total}")